In [50]:
### Adding the sentiment and polarity to each of the tweets

In [51]:
import pandas as pd
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [52]:
tweets = pd.read_csv('DATA/Clean_Tweets_All_Info.csv')


In [53]:
tweets.head(1)

,Unnamed: 0.1,Unnamed: 0,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,mentioned_users,hashtags,emojis,urls,cleaned_text,coords,geometry
0,1,1,1575493060024143874,6160,@_Hermano_7 PSL players would riot 😭,2022-09-29 14:29:51+00:00,0,1,0,0,{'place_id': 'e564d30dc173d2a8'},"Johannesburg, South Africa",['@_Hermano_7'],NaN,['😭'],NaN,psl player would riot,"[27.7518557, -26.5126489, 28.1843404, -26.0396...",POINT (27.968098050000002 -26.276138500000002)


In [54]:
# creating a dataframe with the text as the list
text = tweets['text'].tolist()
text = pd.DataFrame(text, columns = ['Text'])

In [55]:
# this is a cleaning text function... very simular to the one in tweet_cleaning. May just combine this into one function in tweet cleaning. 
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # removes mentions
    text = re.sub(r'#', '', text) # removes #
    text = re.sub(r'RT[\s]+', '', text) # removes RT
    text = re.sub(r'https?:\/\/\S+', '', text) # removes hyper link

    return text
text['Text']=text['Text'].apply(cleanTxt)

In [56]:
# get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# function to compute the negative, neutral and positive analysis
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'
    
text['Subjectivity'] = text['Text'].apply(getSubjectivity)
text['Polarity'] = text['Text'].apply(getPolarity)
text['Analysis'] = text['Polarity'].apply(getAnalysis)


text = text.drop(['Text'], axis = 1)

In [57]:
text

,Subjectivity,Polarity,Analysis
0,0.000000,0.000000,Neutral
1,0.000000,0.000000,Neutral
2,0.500000,0.000000,Neutral
3,0.000000,0.000000,Neutral
4,0.000000,0.000000,Neutral
...,...,...,...
19972,0.933333,0.033333,Positive
19973,0.000000,0.000000,Neutral
19974,0.000000,0.000000,Neutral
19975,0.200000,-0.100000,Negative


In [62]:
# Adding the subjectivity to the tweets

tweets = pd.concat([tweets, text.reindex(tweets.index)], axis = 1)

In [63]:
tweets.head(5)

,Unnamed: 0.1,Unnamed: 0,tweet_id,author_followers,text,created_at,retweets,replies,likes,quote_count,...,mentioned_users,hashtags,emojis,urls,cleaned_text,coords,geometry,Subjectivity,Polarity,Analysis
0,1,1,1575493060024143874,6160,@_Hermano_7 PSL players would riot 😭,2022-09-29 14:29:51+00:00,0,1,0,0,...,['@_Hermano_7'],NaN,['😭'],NaN,psl player would riot,"[27.7518557, -26.5126489, 28.1843404, -26.0396...",POINT (27.968098050000002 -26.276138500000002),0.0,0.0,Neutral
1,2,2,1575489974316584961,1329,Spx yesterday's rally\nPoof gone https://t.co/...,2022-09-29 14:17:35+00:00,0,0,1,0,...,NaN,NaN,NaN,https://t.co/hvAYVEs6NU,spx yesterday's rally\npoof go,"[28.2722463, -26.2315204, 28.4449594, -26.0681...",POINT (28.358602849999997 -26.14985695),0.0,0.0,Neutral
2,3,3,1575482837989474304,1436,@JohnPerlman A driverless car wouldn't work in...,2022-09-29 13:49:14+00:00,1,0,0,0,...,['@JohnPerlman'],['#702drive'],NaN,NaN,driverless car work south africa.the car would...,"[27.9483035, -25.9157727, 28.4198285, -25.5894...",POINT (28.184066 -25.75260515),0.5,0.0,Neutral
3,4,4,1575438290962075648,1881,We used to catch a train from Ikwezi to Mzimhl...,2022-09-29 10:52:13+00:00,1,0,1,0,...,NaN,NaN,NaN,https://t.co/DeR5KoJPXb,use catch train ikwezi mzimhlophe go church in...,"[27.8651453, -26.2359005, 27.9219768, -26.1964...",POINT (27.89356105 -26.2161891),0.0,0.0,Neutral
4,6,6,1575418000047472641,985,"Thunder ⛈ Fire Go Strike You, Fo De Ting You D...",2022-09-29 09:31:35+00:00,0,0,0,0,...,['@wendyshaygh'],NaN,"['⛈', '😭', '🔥', '🔥']",https://t.co/i6Togh9BjD,thunder fire go strike fo de ting,"[27.4580897, -26.5177113, 27.481206, -26.4907865]",POINT (27.46964785 -26.5042489),0.0,0.0,Neutral
